In [2]:
import os
import pandas as pd
import numpy as np
import datetime
import re

#Set the current directory for the kernel
path = os.getcwd()
files = os.listdir(path)
files_xls = [f for f in files if f[-3:] == 'XLS']

#Set the file name pattern to sort the name through the code by date and time rather than sample number
pattern = r'\d{2}_\d{2}_\d{4}\s\d{2}_\d{2}_\d{2}'
filename_datetime_list = []
for filename in files_xls:
    match = re.search(pattern, filename)
    if match:
        datetime_str = match.group()
        datetime_obj = datetime.datetime.strptime(datetime_str, "%d_%m_%Y %H_%M_%S")
        filename_datetime_list.append((filename, datetime_obj))

# Sort the list of tuples based on the datetime object
sorted_filename_datetime_list = sorted(filename_datetime_list, key=lambda x: x[1])
sorted_filenames = [x[0] for x in sorted_filename_datetime_list]

#This part of the code creates a full DataFrame which contains all the data from every excel file by keeping 
#the data sorted by worksheet
dfs = []
for filename in sorted_filenames:
    dfname = pd.ExcelFile(os.path.join(path, filename))
    for items in dfname.sheet_names:
        dfnew = pd.read_excel(dfname, sheet_name=items)
        dfnew['filename'] = filename  # add a new column to identify the source file
        dfs.append(dfnew)
data = pd.concat(dfs, ignore_index=True)

#This part of the script rearranges the data according to the filename and keeps only the information about the 
#second occurrance of Nitrogen, which is the one needed
a = ['Hydrogen', 'Carbon monoxide', 'Nitrogen', 'Methane', 'Ethylene']
data = data.groupby('filename',as_index=False, sort=False).apply(lambda x: x[x['Name'].isin(a)].drop_duplicates(subset='Name', keep='last'))
data = data[['Name', 'Area', 'filename']].reset_index(drop=True)

#UPDATE 1: Dealing with non-numerical data, such as N.D. which means the instrument does not detect the 
#target compound. If this piece of code is not run, the final table will be filled only with null values.
data["Area"] = data["Area"].replace(["N.D."], 0)
data["Area"] = pd.to_numeric(data["Area"])

#Target table generation
pivot_table = data.pivot_table(index='filename', columns='Name', values='Area', sort=False)
column_order = ['Hydrogen', 'Carbon monoxide', 'Nitrogen', 'Methane', 'Ethylene']
pivot_table = pivot_table.reindex(columns=column_order)
pivot_table.to_excel("GC_outputs.xlsx")

In [3]:
pivot_table

Name,Hydrogen,Carbon monoxide,Nitrogen,Methane,Ethylene
filename,,,,,
Sample1_23_02_2023 09_51_37.XLS,88731.0,36852.3,164582.5,0.0,518.6
Sample2_23_02_2023 10_35_49.XLS,83266.6,34808.6,158322.0,1.6,420.2
Sample3_23_02_2023 10_50_52.XLS,111550.7,60494.1,163771.4,24.7,1988.1
Sample4_23_02_2023 11_05_55.XLS,114152.6,58894.2,164939.5,22.4,2017.0
Sample5_23_02_2023 11_20_57.XLS,155033.5,68766.0,168709.6,60.7,4130.6
Sample6_23_02_2023 11_36_01.XLS,158409.4,68089.4,173483.3,51.9,4061.9
Sample7_23_02_2023 11_51_03.XLS,220230.1,71942.9,174227.9,94.6,5984.0
Sample8_23_02_2023 12_06_06.XLS,223446.4,69695.0,180830.9,86.9,5900.0
Sample9_23_02_2023 12_21_08.XLS,292623.4,68335.2,182068.6,144.4,7623.0
